The Jupyter Notebook root path (See "Notebook File Root" in settings) has been set to the workspace folder.
In this case - Masters_6mic_analysis

First, convert your .flac file to .RAW (allowing it to be processed via ODAS)

In [11]:
from jinja2 import Environment, FileSystemLoader
# Jinja allows us to easily fill-out custom text files (e.g., our config file)
import os
import sox

# Specify which file you want to process...
# file_path = "data/IC1/13-05-23/RPiID-000000004e8ff25b/2023-04-18/04-17-09_dur=1200secs.flac"
file_path = "data/SpecificTests/OverlappingBirds_Silwood/6Channels.raw"
folder_path = file_path.split(".")[0]

# Setup a new folder for the processed data
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# CONVERT TO RAW------------------------------------------------------------------------
# Input FLAC file
input_file = file_path

# Output RAW audio file
output_file = folder_path + "/raw_6_chan.raw"

# Create a Sox object
tfm = sox.Transformer()

tfm.set_input_format(rate=16000, channels=6, bits=32, encoding="signed-integer")

tfm.set_output_format(rate=16000, channels=6, bits=32, encoding="signed-integer")

# Convert the FLAC file to RAW audio format
tfm.build(input_file, output_file)

# Print a message when the conversion is done
print("Conversion complete")

Conversion complete


Then, use Jinja to create a new cfg file for that RAW audio segment

In [5]:
# CREATE NEW .cfg FILE------------------------------------------------------------------
# Create a Jinja2 environment with the file system loader
config_path = "src/configs"
env = Environment(loader=FileSystemLoader(config_path))

# Load the template from disk
template = env.get_template("6mic_post_analysis_template.cfg")

# Setup the paths
raw_input_path = output_file

# Render the template with some data
result = template.render(input_path=raw_input_path, folder_path=folder_path)

# # Print the result
# print(result)

# Save the rendered config to file
config_path = folder_path + "/config.cfg"
with open(config_path, "w") as f:
    f.write(result)

Now, run ODAS to retrieve the beamformed / postfiltered audio

**NOTE: There is no Python package for ODAS - it must be run from the command line. Thus, we can use 'subprocess.call' to control this from our Py script

In [10]:
import subprocess
# Allows us to call terminal commands from Python

run_odas_cmd = "../../odas/build/bin/odaslive -c " + config_path

# print(subprocess.call("ls"))
# print(run_odas_cmd)

print("Running ODAS")
subprocess.call(run_odas_cmd, shell=True)       # Python will wait for execution to complete before continuing
# Note: shell=True is important for searching relative file paths
print("ODAS processing complete")


Running ODAS
ODAS processing complete


We then need to convert each channel (of sep and pf) to individual MP3 files, for BirdNET to process

In [9]:
# RAW input files...
sep_input_file = folder_path + "/sep.raw"
pf_input_file = folder_path + "/pf.raw"

for chan_num in range(1, 5):

    # Output MP3 audio files...
    sep_output_file = folder_path + f"/sep_chan{str(chan_num)}.mp3"
    pf_output_file = folder_path + f"/pf_chan{str(chan_num)}.mp3"

    # Create a Sox object
    tfm = sox.Transformer()

    # 4 channels - 1 for each beamformed direction
    tfm.set_input_format(rate=16000, channels=4, bits=32, encoding="signed-integer")    

    tfm.set_output_format(rate=16000, channels=1, bits=32, encoding="signed-integer")

    tfm.channels(chan_num)  # Set channel number to extract

    # Convert the RAW audio files
    tfm.build(sep_input_file, sep_output_file)
    tfm.build(pf_input_file, pf_output_file)